In [16]:
# Import necessary packages
import cv2 as cv
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models

## Detect champion under HUB (without health bar)

In [17]:
# Cut image under the HUB
def cut_image(img):
    crop = []
    for i in range(5):
        x_top = 720 + 95*i
        x_bottom = 720 + 95 + 95*i
        crop.append(img[60:180, x_top:x_bottom])
    return np.asarray(crop) 

In [18]:
# Return bounding boxes under the HUB
def return_box(img,res,loc_draw):
    bb_extend = []
    for i in range(5):
        if res[i] == 1 and loc_draw[i] == True:
            x_top = 720 + 95*i
            bb_extend.append(x_top)
    return bb_extend

In [19]:
# Load model which used to detect cropped images 
def load_model_under_hub():
    # load json and create model
    json_file = open('models/model_under_hub.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("models/model_under_hub.h5")
    return loaded_model
def load_model_health_bar():
    # load json and create model
    json_file = open('models/model_health_bar.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("models/model_health_bar.h5")
    return loaded_model
def load_model_classes():
    # load json and create model
    json_file = open('models/model_classes.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = models.model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights("models/model_classes.h5")
    return loaded_model

In [20]:
# Combine all function to detect champion under the HUB
def return_bb_extend(img,loc_draw,model):
    crops = cut_image(img)
    return return_box(img,model.predict_classes(crops),loc_draw)

## Detect champion with health bar

In [21]:
#  Solve 2 variable equation
def solve_equation(A,B):
    (x_1,y_1) = A
    (x_2,y_2) = B
    a = (y_1 - y_2)/(x_1 - x_2)
    b = y_1 - a*x_1
    return a,b

In [22]:
# Return line equations
def return_lines():
    type_1 = []
    type_2 = []
    for i in range(7):
        x_1 = 1292 - 131*i
        x_2 = 1325 - 143*i
        y_1 = 545
        y_2 = 775
        type_1.append(solve_equation((x_1,y_1),(x_2,y_2)))
    for i in range(7):
        x_1 = 1292 + int(131*0.5) - 131*i
        x_2 = 1325 + int(143*0.5) - 143*i
        y_1 = 545
        y_2 = 775
        type_2.append(solve_equation((x_1,y_1),(x_2,y_2)))
    return (type_1,type_2)

In [23]:
# Return type of champions
def return_type(x,y,data):
    (type_1,type_2) = data
    dis_1 = np.zeros(len(type_1))
    dis_2 = np.zeros(len(type_2))
    for i,para in enumerate(type_1):
        (a,b) = para
        x_hat =  (y - b) / a
        dis_1[i] = abs(x - x_hat)
    for i,para in enumerate(type_2):
        (a,b) = para
        x_hat =  (y - b) / a
        dis_2[i] = abs(x - x_hat)
    return np.min(dis_1)<np.min(dis_2)

In [24]:
def detect_item(img,x,y,w,h,model_health_bar):
    crop = np.asarray([img[y:y+h, x:x+w]])
    return model_health_bar.predict_classes(crop)[0]

In [25]:
# Calculate the height of the bounding box
def cal_height(x,y,w,h,data,img,t):
    if h > 720 and h < 910:
        return 910
    if h <= 105:
        return 180
    h = h + 50 + (t==3)*10
    if x < 560 and y >= 120 and y <= 175:
        return 250
    if h <= 720 or h > 105:
        if (return_type(x + 0.5*w,h+30,data)):
            type_1 = [690,520,370,250]
            height = np.array(type_1)
            height = height - h
            height_list = type_1
        else:
            type_2 = [780,600,450,325]
            height = np.array(type_2)
            height = height - h
            height_list = type_2
        if height[height>0].shape[0] > 0:
            return height_list[np.argmin(height[height>0])]
        else:
            return "None"

In [26]:
# Remove duplicate bounding box
from scipy.spatial.distance import cdist
def remove_bb(x,y,t,prob):
    prob = np.array(prob)
    m = len(x)
    select = [i for i in range(m)]
    result = []
    A = np.array(np.vstack((x, y)).T)
    matrix_A = np.array(A)
    dis = cdist(matrix_A,matrix_A)
    is_near = dis<10
    for i in select:
        cluster = np.where(is_near[i] == True)[0].tolist()
        result.append(cluster[np.argmax(prob[cluster])])
    result = list(dict.fromkeys(result))
    return zip(np.array(x)[result],np.array(y)[result],np.array(t)[result])

In [27]:
def create_dict():
    dictionary = {}
    with open("classes.txt") as f:
        content = f.readlines()
    for i,line in enumerate(content):
        dictionary[line[0:-1]] = i
    return dictionary

## Main

In [28]:
def process(name,model_under_hub,model_health_bar,model_classes,data,champions):
    img_rgb = cv.imread("images/"+ name +".png")
    img_rgb = cv.resize(img_rgb,(1920,1200))
    img_gray = cv.cvtColor(img_rgb, cv.COLOR_BGR2GRAY)
    img_copy = img_rgb.copy()

    bound_boxes = []
    prob = []
    bbs_store = []
    loc_draw =[True,True,True,True,True]

    template = cv.imread('templates/tier1.png',0)
    wo, ho = template.shape[::-1]
    res = cv.matchTemplate(img_gray,template,cv.TM_CCORR_NORMED)
    threshold = 0.92
    loc_1 = np.where( res >= threshold)
    loc_1_prob = res[np.where( res >= threshold)]
    prob.extend(loc_1_prob)
    for pt in zip(*loc_1[::-1]):
        pt = list(pt)
        pt.append(1)
        bound_boxes.append(pt)
        
    template = cv.imread('templates/tier2.png',0)
    wo, ho = template.shape[::-1]
    res = cv.matchTemplate(img_gray,template,cv.TM_CCORR_NORMED)
    threshold = 0.92
    loc_2 = np.where( res >= threshold)
    loc_2_prob = res[np.where( res >= threshold)]
    prob.extend(loc_2_prob)
    for pt in zip(*loc_2[::-1]):
        pt = list(pt)
        pt.append(2)
        bound_boxes.append(pt)

    template = cv.imread('templates/tier3.png',0)
    wo, ho = template.shape[::-1]
    res = cv.matchTemplate(img_gray,template,cv.TM_CCORR_NORMED)
    threshold = 0.92
    loc_3 = np.where( res >= threshold)
    loc_3_prob = res[np.where( res >= threshold)]
    prob.extend(loc_3_prob)
    for pt in zip(*loc_3[::-1]):
        pt = list(pt)
        pt.append(3)
        bound_boxes.append(pt) 

    cor = list(zip(*bound_boxes))
    loc_x = list(cor[0])
    loc_y = list(cor[1])
    tier = list(cor[2])
    bb = remove_bb(loc_x,loc_y,tier,prob)
    
    for pt in bb:
        x = pt[0]-5
        y = pt[1]-5
        w = wo
        i = detect_item(img_rgb,x,y+20,w,20,model_health_bar)
        h = cal_height(x,y,w,pt[1] + ho + 20*i,data,img_rgb,pt[2])
        
        if h != "None":
            if h == 180 and x > 720 and x < 1195:
                h = h - pt[1]
                cv.rectangle(img_copy, (x,y), (x + w, y + h), (0,0,255), 2)
                loc = (x - 720) // 95 
                loc_draw[loc] = False
                crop = np.asarray([cv.resize(img_rgb[y+15+i*20:y+h, x:x+w],(80,120))])
                champion = model_classes.predict_classes(crop/255)[0]
                cv.putText(img_copy, champions[champion] + " " + str(pt[2]) + "*", (pt[0], pt[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
            else:  
                h = h - pt[1]
                cv.rectangle(img_copy, (x,y), (x + w, y + h), (0,0,255), 2)
                crop = np.asarray([cv.resize(img_rgb[y+15+i*20:y+h, x:x+w],(80,120))])
                champion = model_classes.predict_classes(crop/255)[0]
                cv.putText(img_copy, champions[champion] + " " + str(pt[2]) + "*", (pt[0], pt[1]-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
            
    bb_extend = return_bb_extend(img_rgb,loc_draw,model_under_hub)
    for x in bb_extend:
        y = 60
        w = 95
        h = 120
        i = detect_item(img_rgb,x + 47 - 42,y+20,84,20,model_health_bar)
        cv.rectangle(img_copy, (x,y), (x + w, y + h), (0,0,255), 2)
        crop = np.asarray([cv.resize(img_rgb[y+15+i*20:y+h, x:x+w],(80,120))])
        champion = model_classes.predict_classes(crop/255)[0]
        cv.putText(img_copy, champions[champion] + " ", (x+5, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.5, (36,255,12), 2)
    
    cv.imwrite("results/"+name+".png",img_copy)
#     cv.imwrite("debug.png",img_copy)

In [29]:
from os import listdir
model_under_hub = load_model_under_hub()
model_health_bar = load_model_health_bar()
model_classes = load_model_classes()
data = return_lines()
champions = list(create_dict().keys())

In [30]:
for name in listdir("images"):
    name = name[:-4]
    process(name,model_under_hub,model_health_bar,model_classes,data,champions)